In [94]:
import pickle
from datetime import datetime
from pprint import pp

import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split

In [95]:
SAVE_DATA = True

In [96]:
class MyUtil:
    def save_data(filename, data):
        with open(filename, "wb") as file:
            pickle.dump(data, file)

    def load_data(filename):
        with open(filename, "rb") as file:
            data = pickle.load(file)
        return data

    def get_dt():
        return datetime.now().strftime("%Y-%m-%d_%H-%M")

In [97]:
class DataHandler:
    def __init__(self, _X, _Y, scalerX, scalerY):
        self._X = _X
        self._Y = _Y
        self.scalerX = scalerX
        self.scalerY = scalerY
        self.X_train = None
        self.X_test = None
        self.Y_train = None
        self.Y_test = None

    def split_and_scale(self, test_size, random_state):
        _X_train, _X_test, _Y_train, _Y_test = train_test_split(
            self._X, self._Y, test_size=test_size, random_state=random_state
        )
        self.X_train = self.scalerX.fit_transform(_X_train)
        self.X_test = self.scalerX.transform(_X_test)

        self.Y_train = self.scalerY.fit_transform(_Y_train)
        self.Y_test = self.scalerY.transform(_Y_test)

    def get_train(self):
        return self.X_train, self.Y_train

    def get_test(self):
        return self.X_test, self.Y_test

In [98]:
class RegSwitcher(BaseEstimator):
    def __init__(self, base):
        self.base = base

    def fit(self, X, Y):
        self.base.fit(X, Y)
        self.is_fitted_ = True
        return self

    def predict(self, X):
        return self.base.predict(X)

In [99]:
dt = MyUtil.get_dt()
print(dt)

2025-05-16_10-43


In [100]:
df = pd.read_excel("data.xlsx", index_col="exp")
df.head()

,m1,m2,m3,s1__autocorrelation__lag_8,s1__autocorrelation__lag_9,s1__autocorrelation__lag_7,s1__autocorrelation__lag_6,s1__autocorrelation__lag_5,s1__autocorrelation__lag_4,s1__longest_strike_above_mean,...,s1__ar_coefficient__coeff_3__k_10,s1__approximate_entropy__m_2__r_0.1,s1__lempel_ziv_complexity__bins_3,s1__partial_autocorrelation__lag_4,"s1__fft_coefficient__attr_""abs""__coeff_7","s1__agg_autocorrelation__f_agg_""var""__maxlag_40",s1__spkt_welch_density__coeff_2,y1,y2,y3
exp,,,,,,,,,,,,,,,,,,,,,
E001,150.223716,1176.177278,1.142097,-0.305434,-0.519191,-0.074829,0.159896,0.385790,0.590387,14,...,0.183996,0.158567,0.204152,-0.360084,0.293617,0.499488,5.886812e-08,55.460434,1.065917,114.578620
E002,102.534268,1483.654982,1.104716,-0.243785,-0.454262,-0.021002,0.202836,0.416423,0.608972,14,...,0.184370,0.144742,0.203008,-0.344364,6.142373,0.477743,3.643621e-06,50.640306,1.285666,124.651484
E003,119.890549,1254.897451,2.162773,-0.329006,-0.543405,-0.095913,0.142612,0.373002,0.582151,14,...,0.184036,0.144268,0.208163,-0.364611,26.783283,0.506435,1.590028e-04,50.832405,1.154859,57.018054
E004,162.830799,1302.043195,1.308283,-0.065152,-0.266498,0.138913,0.337187,0.521401,0.683873,16,...,0.187213,0.137326,0.193662,-0.355441,33.227591,0.460547,7.926165e-04,62.476545,1.025161,132.221218
E005,165.720956,1154.482314,1.566830,-0.304881,-0.518177,-0.074836,0.159321,0.384728,0.589003,14,...,0.183978,0.128546,0.192440,-0.357588,11.439470,0.497940,1.462831e-04,57.634438,1.043776,92.160269


In [101]:
_X = df.iloc[:, :-3].values
_Y = df.iloc[:, -3:].values
print(_X.shape)
print(_Y.shape)

(100, 47)
(100, 3)


In [102]:
from sklearn.preprocessing import StandardScaler

data_handler = DataHandler(
    _X=_X, _Y=_Y, scalerX=StandardScaler(), scalerY=StandardScaler()
)

In [103]:
# data_handler.split_and_scale(test_size=0.2, random_state=6)
# X_train, Y_train = data_handler.get_train()
# print(X_train.shape, Y_train.shape)

In [104]:
# Model hyper parameters
from sklearn.model_selection import ParameterGrid

param_grid_split = [{"random_state": [1, 2, 3, 4, 5], "test_size": [0.3]}]

param_list_split = list(ParameterGrid(param_grid_split))


In [105]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

base_lr = MultiOutputRegressor(LinearRegression())
base_svr = MultiOutputRegressor(SVR())
base_rf = MultiOutputRegressor(RandomForestRegressor())

reg = RegSwitcher(base=base_rf)

pp(reg.get_params())

{'base__estimator__bootstrap': True,
 'base__estimator__ccp_alpha': 0.0,
 'base__estimator__criterion': 'squared_error',
 'base__estimator__max_depth': None,
 'base__estimator__max_features': 1.0,
 'base__estimator__max_leaf_nodes': None,
 'base__estimator__max_samples': None,
 'base__estimator__min_impurity_decrease': 0.0,
 'base__estimator__min_samples_leaf': 1,
 'base__estimator__min_samples_split': 2,
 'base__estimator__min_weight_fraction_leaf': 0.0,
 'base__estimator__monotonic_cst': None,
 'base__estimator__n_estimators': 100,
 'base__estimator__n_jobs': None,
 'base__estimator__oob_score': False,
 'base__estimator__random_state': None,
 'base__estimator__verbose': 0,
 'base__estimator__warm_start': False,
 'base__estimator': RandomForestRegressor(),
 'base__n_jobs': None,
 'base': MultiOutputRegressor(estimator=RandomForestRegressor())}


In [106]:
param_grid_hyper = [
    {"base": [base_lr]},
    {"base": [base_svr], "base__estimator__C": [0.01, 0.1, 1]},
    {"base": [base_rf], "base__estimator__n_estimators": [10, 50, 200]},
]

In [107]:
# Initialize blank model (optional)
reg = RegSwitcher(base=None)

df_arr = []
for idx_split, param_split in enumerate(param_list_split):
    data_handler.split_and_scale(**param_split)
    X_train, Y_train = data_handler.get_train()
    
    gs = GridSearchCV(
        estimator=reg,
        param_grid=param_grid_hyper,
        cv=3,
        # scoring="neg_mean_squared_error",
        scoring="r2",
        n_jobs=-1,
    )
    gs.fit(X_train, Y_train)
    _df = pd.DataFrame(gs.cv_results_)
    _df["id_split"] = idx_split
    _df["param_split"] = [param_split for _ in range(_df.shape[0])]
    df_arr.append(_df)

df_fit = pd.concat(df_arr)
df_fit = df_fit.reset_index().rename(columns={"index": "id_gs"})

In [108]:
df_fit

,id_gs,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base,param_base__estimator__C,param_base__estimator__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,id_split,param_split
0,0,0.014071,0.001767,0.002347,2.447175e-04,MultiOutputRegressor(estimator=LinearRegressio...,NaN,NaN,{'base': MultiOutputRegressor(estimator=Linear...,0.038175,0.752024,-7.906865,-2.372222,3.924419,7,0,"{'random_state': 1, 'test_size': 0.3}"
1,1,0.009386,0.000477,0.004675,4.771235e-04,MultiOutputRegressor(estimator=SVR()),0.01,NaN,{'base': MultiOutputRegressor(estimator=SVR())...,-0.002276,-0.041889,0.024655,-0.006503,0.027330,6,0,"{'random_state': 1, 'test_size': 0.3}"
2,2,0.007714,0.002334,0.005696,2.781810e-03,MultiOutputRegressor(estimator=SVR()),0.10,NaN,{'base': MultiOutputRegressor(estimator=SVR())...,0.302295,0.309570,0.302825,0.304896,0.003312,5,0,"{'random_state': 1, 'test_size': 0.3}"
3,3,0.011382,0.001723,0.005882,1.302470e-03,MultiOutputRegressor(estimator=SVR()),1.00,NaN,{'base': MultiOutputRegressor(estimator=SVR())...,0.617707,0.814445,0.800537,0.744229,0.089645,4,0,"{'random_state': 1, 'test_size': 0.3}"
4,4,0.099902,0.009578,0.006361,1.558638e-03,MultiOutputRegressor(estimator=RandomForestReg...,NaN,10.0,{'base': MultiOutputRegressor(estimator=Random...,0.736135,0.851747,0.757178,0.781687,0.050280,1,0,"{'random_state': 1, 'test_size': 0.3}"
5,5,0.397632,0.043272,0.016054,4.432591e-03,MultiOutputRegressor(estimator=RandomForestReg...,NaN,50.0,{'base': MultiOutputRegressor(estimator=Random...,0.713281,0.848118,0.735107,0.765502,0.059094,3,0,"{'random_state': 1, 'test_size': 0.3}"
6,6,1.147407,0.060091,0.034565,2.605142e-03,MultiOutputRegressor(estimator=RandomForestReg...,NaN,200.0,{'base': MultiOutputRegressor(estimator=Random...,0.727257,0.866232,0.729568,0.774352,0.064975,2,0,"{'random_state': 1, 'test_size': 0.3}"
7,0,0.003336,0.000476,0.003844,6.248409e-04,MultiOutputRegressor(estimator=LinearRegressio...,NaN,NaN,{'base': MultiOutputRegressor(estimator=Linear...,-0.139373,0.777359,-6.449578,-1.937197,3.212609,7,1,"{'random_state': 2, 'test_size': 0.3}"
8,1,0.007687,0.000634,0.001514,4.121634e-04,MultiOutputRegressor(estimator=SVR()),0.01,NaN,{'base': MultiOutputRegressor(estimator=SVR())...,0.042289,0.032906,-0.041063,0.011377,0.037278,6,1,"{'random_state': 2, 'test_size': 0.3}"
9,2,0.005526,0.001632,0.003686,1.192138e-03,MultiOutputRegressor(estimator=SVR()),0.10,NaN,{'base': MultiOutputRegressor(estimator=SVR())...,0.328093,0.341163,0.244914,0.304724,0.042627,5,1,"{'random_state': 2, 'test_size': 0.3}"


In [109]:
df_fit["estimator"] = df_fit["param_base"].apply(
    lambda x: x.estimator.__class__.__name__
)

In [110]:
pp(df_fit.columns)

cols = [
    "id_split",
    "param_split",
    "id_gs",
    "params",
    "estimator",
    "mean_test_score",
    "std_test_score",
    "rank_test_score",
]

df_fit = df_fit[cols]
df_fit

Index(['id_gs', 'mean_fit_time', 'std_fit_time', 'mean_score_time',
       'std_score_time', 'param_base', 'param_base__estimator__C',
       'param_base__estimator__n_estimators', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score', 'id_split', 'param_split',
       'estimator'],
      dtype='object')


,id_split,param_split,id_gs,params,estimator,mean_test_score,std_test_score,rank_test_score
0,0,"{'random_state': 1, 'test_size': 0.3}",0,{'base': MultiOutputRegressor(estimator=Linear...,LinearRegression,-2.372222,3.924419,7
1,0,"{'random_state': 1, 'test_size': 0.3}",1,{'base': MultiOutputRegressor(estimator=SVR())...,SVR,-0.006503,0.027330,6
2,0,"{'random_state': 1, 'test_size': 0.3}",2,{'base': MultiOutputRegressor(estimator=SVR())...,SVR,0.304896,0.003312,5
3,0,"{'random_state': 1, 'test_size': 0.3}",3,{'base': MultiOutputRegressor(estimator=SVR())...,SVR,0.744229,0.089645,4
4,0,"{'random_state': 1, 'test_size': 0.3}",4,{'base': MultiOutputRegressor(estimator=Random...,RandomForestRegressor,0.781687,0.050280,1
5,0,"{'random_state': 1, 'test_size': 0.3}",5,{'base': MultiOutputRegressor(estimator=Random...,RandomForestRegressor,0.765502,0.059094,3
6,0,"{'random_state': 1, 'test_size': 0.3}",6,{'base': MultiOutputRegressor(estimator=Random...,RandomForestRegressor,0.774352,0.064975,2
7,1,"{'random_state': 2, 'test_size': 0.3}",0,{'base': MultiOutputRegressor(estimator=Linear...,LinearRegression,-1.937197,3.212609,7
8,1,"{'random_state': 2, 'test_size': 0.3}",1,{'base': MultiOutputRegressor(estimator=SVR())...,SVR,0.011377,0.037278,6
9,1,"{'random_state': 2, 'test_size': 0.3}",2,{'base': MultiOutputRegressor(estimator=SVR())...,SVR,0.304724,0.042627,5


In [111]:
if SAVE_DATA:
    filename = f"S04_data_{dt}.pkl"

    data_save = {
        "desc": "This is the saved data",
        "data_handler": data_handler,
        "param_split": param_split,
        "param_grid_hyper": param_grid_hyper,
        "df_fit": df_fit,
    }

    # Save the model
    MyUtil.save_data(filename=filename, data=data_save)